**Course:**     Natural Language Processing Lab (NLP701B)

**Name:**       Sarfraz Ahmad (24010739)

**Institution:** Mohamed Bin Zayed University of Artificial Intelligence

**Email:** sarfraz.ahmad@mbzuai.ac.ae

**Instructor:** Prof. Ted Briscoe / Prof. Preslav Nakov

**Module:**     Assignment # 2: Multilingual Characterization and Extraction of Narratives from Online News

**Team ID:** Sarfraz307

**Repo:** [GitHub](https://github.com/SarfrazAhmad307/Assignment2-NLP701)

In [1]:
import pandas as pd
import logging
import re
import nltk
import os
import string
import json
import emoji

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import plotly.express as px

import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
import nltk
import transformers
import torch
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
#import bert
import tensorflow as tf

from numpy import array
from numpy import asarray
from numpy import zeros

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import LabelPowerset

from bs4 import BeautifulSoup

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW

from wordcloud import WordCloud

from IPython.display import Image

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
%matplotlib inline

# Ensure you have the stopwords and punkt tokenizer models
nltk.download('stopwords')
nltk.download('punkt')

from bs4 import BeautifulSoup
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sarfrazahmad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sarfrazahmad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sarfrazahmad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sarfrazahmad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sarfrazahmad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data Preprocessing

In [2]:
def read_txt_filef(file_path, data_type):
    # Initialize lists to store data
    article_ids = []
    entity_mentions = []
    start_offsets = []
    end_offsets = []
    main_roles = []
    fine_grained_roles = []
    
    # Read the txt file
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')  # Split each row by tab
            
            article_ids.append(parts[0])  # article_id
            entity_mentions.append(parts[1])  # entity_mention
            start_offsets.append(int(parts[2]))  # start_offset
            end_offsets.append(int(parts[3]))  # end_offset

            if data_type == 'train':
                main_roles.append(parts[4])  # main_role
                
                # Handle multiple fine_grained_roles (split by space if there are multiple roles)
                fine_grained_roles.append(parts[5:])  # fine-grained_roles (as list)
            else:
                continue
    
    if data_type == 'train':
        # Create a DataFrame
        df = pd.DataFrame({
            'article_id': article_ids,
            'entity_mention': entity_mentions,
            'start_offset': start_offsets,
            'end_offset': end_offsets,
            'main_role': main_roles,
            'fine_grained_roles': fine_grained_roles
        })
    else:
        # Create a DataFrame
        df = pd.DataFrame({
            'article_id': article_ids,
            'entity_mention': entity_mentions,
            'start_offset': start_offsets,
            'end_offset': end_offsets,
        })


    return df

In [3]:
# Read the txt file
df = read_txt_filef('data/target_4_December_release/EN/subtask-1-annotations.txt', 'train')
df.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles
0,EN_CC_100013.txt,Bill Gates,93,102,Antagonist,"[Deceiver, Corrupt]"
1,EN_CC_100013.txt,BBC,1860,1862,Antagonist,[Deceiver]
2,EN_CC_100013.txt,Jeffrey Epstein,2005,2019,Antagonist,[Corrupt]
3,EN_UA_300009.txt,Fail Alsynov,176,187,Protagonist,"[Rebel, Martyr]"
4,EN_UA_300009.txt,Bashkir people,1616,1629,Innocent,[Victim]


In [4]:
df_test = read_txt_filef('data/dev-documents_4_December/EN/subtask-1-entity-mentions.txt', 'test')
df_test.head()

,article_id,entity_mention,start_offset,end_offset
0,EN_UA_DEV_100012.txt,Washington,1441,1450
1,EN_UA_DEV_100012.txt,Americans,2397,2405
2,EN_CC_200053.txt,Rishi Sunak,85,95
3,EN_CC_200040.txt,Extinction Rebellion,189,208
4,EN_CC_200070.txt,Eric Garcetti,89,101


In [5]:
def additional_columns(df, data_type):
    # read the txt files based on the article_ids and store as document column
    documents = []

    for article_id in df['article_id'].unique():
        if data_type == 'train':
            with open(f'data/target_4_December_release/EN/raw-documents/{article_id}', 'r') as file:
                documents.append(file.read())
        else:
            with open(f'data/dev-documents_4_December/EN/subtask-1-documents/{article_id}', 'r') as file:
                documents.append(file.read())

    # Create a DataFrame
    df_documents = pd.DataFrame({
        'article_id': df['article_id'].unique(),
        'document': documents
    })

    # add the document column to the main DataFrame
    df = df.merge(df_documents, on='article_id')

    # check the entity_mention in document based on start_offset and end_offset
    df['document_entity_mention'] = df.apply(lambda x: x['document'][x['start_offset']:x['end_offset']+1], axis=1)

    # now we have got the entity_mention in the document, we can check if the entity_mention is same as the entity_mention in the data
    df['entity_mention_match'] = df['entity_mention'] == df['document_entity_mention']

    if data_type == 'train':
        # rearrange the columns
        df = df[['article_id', 'document', 'entity_mention', 'document_entity_mention', 'entity_mention_match', 
                 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles']]
    else:
        # rearrange the columns
        df = df[['article_id', 'document', 'entity_mention', 'document_entity_mention', 'entity_mention_match', 
                 'start_offset', 'end_offset']]

    # drop the document_entity_mention and article_id columns
    # df.drop(columns=['article_id'], inplace=True)

    return df

In [6]:
df = additional_columns(df, 'train')
df_test = additional_columns(df_test, 'test')
df.head()

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles
0,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Bill Gates,Bill Gates,True,93,102,Antagonist,"[Deceiver, Corrupt]"
1,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,BBC,BBC,True,1860,1862,Antagonist,[Deceiver]
2,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Jeffrey Epstein,Jeffrey Epstein,True,2005,2019,Antagonist,[Corrupt]
3,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Fail Alsynov,Fail Alsynov,True,176,187,Protagonist,"[Rebel, Martyr]"
4,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Bashkir people,Bashkir people,True,1616,1629,Innocent,[Victim]


In [7]:
df_test.head()

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset
0,EN_UA_DEV_100012.txt,Ukraine's Minerals: What the West is Fighting ...,Washington,Washington,True,1441,1450
1,EN_UA_DEV_100012.txt,Ukraine's Minerals: What the West is Fighting ...,Americans,Americans,True,2397,2405
2,EN_CC_200053.txt,UK’s Sunak Reverses Decision to Skip COP27 Cli...,Rishi Sunak,Rishi Sunak,True,85,95
3,EN_CC_200040.txt,Climate Protesters Out Of Control As They Atta...,Extinction Rebellion,Extinction Rebellion,True,189,208
4,EN_CC_200070.txt,Wat? L.A. Mayor Garcetti Flies to Argentina to...,Eric Garcetti,Eric Garcetti,True,89,101


In [13]:
# print rows where entinty_mention_match is False
df[df['entity_mention_match'] == False]

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles
63,EN_UA_002668.txt,Putin may ABANDON siege of Kyiv and try to bli...,Russia,ussia,False,1589,1594,Antagonist,[Terrorist]
64,EN_UA_002668.txt,Putin may ABANDON siege of Kyiv and try to bli...,Putin,utin,False,1750,1754,Antagonist,[Incompetent]
469,EN_UA_024050.txt,How NATO powers are using the U.K.’s World War...,Germany,rmany h,False,2178,2184,Antagonist,[Traitor]
470,EN_UA_024050.txt,How NATO powers are using the U.K.’s World War...,Ukraine,raine t,False,2252,2258,Antagonist,[Incompetent]
471,EN_UA_024050.txt,How NATO powers are using the U.K.’s World War...,France,ance a,False,2529,2534,Antagonist,[Conspirator]
472,EN_UA_024050.txt,How NATO powers are using the U.K.’s World War...,Poland,"land,",False,2540,2545,Antagonist,[Conspirator]
473,EN_UA_024050.txt,How NATO powers are using the U.K.’s World War...,Weimar Triangle,imar Triangle w,False,2902,2916,Antagonist,[Conspirator]
553,EN_UA_001032.txt,Russia-Ukraine war map: Where are Russian troo...,Russia,ussia,False,2064,2069,Antagonist,[Instigator]
554,EN_UA_001032.txt,Russia-Ukraine war map: Where are Russian troo...,Ukraine,kraine?,False,2079,2085,Innocent,[Victim]
555,EN_UA_001032.txt,Russia-Ukraine war map: Where are Russian troo...,Joe Biden,oe Biden,False,2609,2617,Protagonist,[Guardian]


In [14]:
# print rows where entinty_mention_match is False
df_test[df_test['entity_mention_match'] == False]

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset


In [15]:
def fix_offsets(df):
    # for rows where entity_mention_match is False, subtract 2 from start_offset and end_offset
    df.loc[df['entity_mention_match'] == False, 'start_offset'] -= 1
    df.loc[df['entity_mention_match'] == False, 'end_offset'] -= 1

    # Now we can check if the entity_mention is same as the entity_mention in the data
    df['document_entity_mention'] = df.apply(lambda x: x['document'][x['start_offset']:x['end_offset']+1], axis=1)
    df['entity_mention_match'] = df['entity_mention'] == df['document_entity_mention']

    df.loc[df['entity_mention_match'] == False, 'start_offset'] -= 1
    df.loc[df['entity_mention_match'] == False, 'end_offset'] -= 1

    # Now we can check if the entity_mention is same as the entity_mention in the data
    df['document_entity_mention'] = df.apply(lambda x: x['document'][x['start_offset']:x['end_offset']+1], axis=1)
    df['entity_mention_match'] = df['entity_mention'] == df['document_entity_mention']

    return df

In [16]:
df = fix_offsets(df)
df_test = fix_offsets(df_test)

In [17]:
# print rows where entinty_mention_match is False
df[df['entity_mention_match'] == False]

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles


In [18]:
# print rows where entinty_mention_match is False
df_test[df_test['entity_mention_match'] == False]

,article_id,document,entity_mention,document_entity_mention,entity_mention_match,start_offset,end_offset


In [19]:
# drop the document_entity_mention and article_id columns
df.drop(columns=['document_entity_mention'], inplace=True)
df_test.drop(columns=['document_entity_mention'], inplace=True)

In [20]:
df['Len of classes'] = df['fine_grained_roles'].apply(lambda x: len(x))
df.head()

,article_id,document,entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles,Len of classes
0,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Bill Gates,True,93,102,Antagonist,"[Deceiver, Corrupt]",2
1,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,BBC,True,1860,1862,Antagonist,[Deceiver],1
2,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Jeffrey Epstein,True,2005,2019,Antagonist,[Corrupt],1
3,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Fail Alsynov,True,176,187,Protagonist,"[Rebel, Martyr]",2
4,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Bashkir people,True,1616,1629,Innocent,[Victim],1


In [21]:
# Print rows where len of classes is greater than 1
df[df['Len of classes'] > 1].head()

,article_id,document,entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles,Len of classes
0,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Bill Gates,True,93,102,Antagonist,"[Deceiver, Corrupt]",2
3,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Fail Alsynov,True,176,187,Protagonist,"[Rebel, Martyr]",2
5,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Bashkort,True,1791,1798,Protagonist,"[Rebel, Guardian]",2
47,EN_UA_300129.txt,The Kremlin Is Setting Up WW III by Sitting on...,the U.S,True,591,597,Antagonist,"[Instigator, Foreign Adversary]",2
51,EN_UA_002531.txt,Kyiv's Mayor Says He's 'Ready to Fight' as Rus...,Kyiv Mayor Vitali Klitschko,True,76,102,Protagonist,"[Rebel, Martyr]",2


In [22]:
# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the column
binary_labels = mlb.fit_transform(df['fine_grained_roles'])

# Create a new DataFrame with binary labels
binary_labels_df = pd.DataFrame(binary_labels, columns=mlb.classes_)

# Concatenate the original dataframe with the new binary columns
df = pd.concat([df, binary_labels_df], axis=1)

# Display the updated DataFrame
df.head()

,article_id,document,entity_mention,entity_mention_match,start_offset,end_offset,main_role,fine_grained_roles,Len of classes,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Bill Gates,True,93,102,Antagonist,"[Deceiver, Corrupt]",2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,BBC,True,1860,1862,Antagonist,[Deceiver],1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Jeffrey Epstein,True,2005,2019,Antagonist,[Corrupt],1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Fail Alsynov,True,176,187,Protagonist,"[Rebel, Martyr]",2,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Bashkir people,True,1616,1629,Innocent,[Victim],1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [23]:
df.drop(columns=['Len of classes', 'entity_mention_match'], inplace=True)

In [24]:
df_test.drop(columns=['entity_mention_match'], inplace=True)

In [25]:
df.head()

,article_id,document,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Bill Gates,93,102,Antagonist,"[Deceiver, Corrupt]",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,BBC,1860,1862,Antagonist,[Deceiver],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,EN_CC_100013.txt,Bill Gates Says He Is ‘The Solution’ To Climat...,Jeffrey Epstein,2005,2019,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Fail Alsynov,176,187,Protagonist,"[Rebel, Martyr]",0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,EN_UA_300009.txt,Russia: Clashes erupt in Bashkortostan as righ...,Bashkir people,1616,1629,Innocent,[Victim],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [26]:
df_test.head()

,article_id,document,entity_mention,start_offset,end_offset
0,EN_UA_DEV_100012.txt,Ukraine's Minerals: What the West is Fighting ...,Washington,1441,1450
1,EN_UA_DEV_100012.txt,Ukraine's Minerals: What the West is Fighting ...,Americans,2397,2405
2,EN_CC_200053.txt,UK’s Sunak Reverses Decision to Skip COP27 Cli...,Rishi Sunak,85,95
3,EN_CC_200040.txt,Climate Protesters Out Of Control As They Atta...,Extinction Rebellion,189,208
4,EN_CC_200070.txt,Wat? L.A. Mayor Garcetti Flies to Argentina to...,Eric Garcetti,89,101


In [27]:
# Function to get the paragraph containing the entity mention
def get_paragraph_containing_entity(doc, start_offset, end_offset):
    # Extract the text before and after the entity mention using the offsets
    # Find the start of the paragraph by moving backwards from the start_offset
    start_paragraph = doc.rfind('\n\n', 0, start_offset)
    if start_paragraph == -1:
        start_paragraph = 0  # If no paragraph break found, assume start of document
    
    # Find the end of the paragraph by moving forwards from the end_offset
    end_paragraph = doc.find('\n\n', end_offset)
    if end_paragraph == -1:
        end_paragraph = len(doc)  # If no paragraph break found, assume end of document
    
    # Extract the full paragraph containing the entity
    paragraph = doc[start_paragraph:end_paragraph].strip()
    
    return paragraph

# Get the surrounding sentences for each row
df["entity_paragraph"] = df.apply(lambda x: get_paragraph_containing_entity(x['document'], x['start_offset'], x['end_offset']), axis=1)
df_test["entity_paragraph"] = df_test.apply(lambda x: get_paragraph_containing_entity(x['document'], x['start_offset'], x['end_offset']), axis=1)

In [28]:
# Rearrange the columns document, entity_mention, surrounding_span, start_offset, end_offset, main_role, fine_grained_roles, and one-hot encoded columns
df.drop(columns=['document'], inplace=True)
df_test.drop(columns=['document'], inplace=True)

df.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous,entity_paragraph
0,EN_CC_100013.txt,Bill Gates,93,102,Antagonist,"[Deceiver, Corrupt]",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bill Gates has the right to fly around the wor...
1,EN_CC_100013.txt,BBC,1860,1862,Antagonist,[Deceiver],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,While the BBC interview was set up to look lik...
2,EN_CC_100013.txt,Jeffrey Epstein,2005,2019,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,While the BBC interview was set up to look lik...
3,EN_UA_300009.txt,Fail Alsynov,176,187,Protagonist,"[Rebel, Martyr]",0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,Russian riot police clashed with protesters in...
4,EN_UA_300009.txt,Bashkir people,1616,1629,Innocent,[Victim],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,Alsynov contends the Bashkir words meant “poor...


In [29]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,Washington “cannot afford” to allow Russia to ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,"""The fate of the colonies is always the same: ..."
2,EN_CC_200053.txt,Rishi Sunak,85,95,The UK’s Prime Minister Rishi Sunak on Wednesd...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,"Those ""Extinction Rebellion"" activists everyon..."
4,EN_CC_200070.txt,Eric Garcetti,89,101,Los Angeles Mayor Eric Garcetti joined a “glob...


In [30]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text


def clean_contractions(text):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text)
    text = remove_space(text)
    # text = remove_stopwords(text)  # Added stopword removal step
    return text

In [31]:
df['entity_paragraph'] = df['entity_paragraph'].apply(text_preprocessing_pipeline)
df_test['entity_paragraph'] = df_test['entity_paragraph'].apply(text_preprocessing_pipeline)

/var/folders/26/9210jd411_1533jzd44vyvz80000gn/T/ipykernel_30762/3386016250.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'lxml').get_text()


In [32]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford to allow russia to ac...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,the fate of the colonies is always the same th...
2,EN_CC_200053.txt,Rishi Sunak,85,95,the uk s prime minister rishi sunak on wednesd...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,those extinction rebellion activists everyone ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined a globa...


In [33]:
def plot_most_frequent_words(df, text_column, top_n=20):
    '''
    Function to plot the most frequent words in a specified text column of the DataFrame.
    
    Parameters:
    - df: DataFrame containing the text data.
    - text_column: Name of the column in the DataFrame to process.
    - top_n: Number of most frequent words to display (default is 10).
    '''
    
    # Function to clean and tokenize the text
    def preprocess_text(text):
        tokens = word_tokenize(text)  # Tokenize the text
        return tokens

    # Flatten the list of tokens from the text column and count word frequencies
    all_tokens = [token for text in df[text_column] for token in preprocess_text(text)]
    word_counts = Counter(all_tokens)

    # Get the most common words and their frequencies
    most_common_words = word_counts.most_common(top_n)

    # Create a DataFrame for the plot
    word_freq_df = pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])

    # Create an interactive bar chart using Plotly
    fig = px.bar(word_freq_df, 
                 x='Word', 
                 y='Frequency', 
                 title=f"Most Frequent Words in {text_column.capitalize()} Column",
                 labels={'Word': 'Words', 'Frequency': 'Occurrences'},
                 color='Frequency',  # Optional: Adds color to the bars based on frequency
                 template='plotly_dark')  # Optional: Dark theme

    # Show the plot
    fig.show()

# Plot the most frequent words in the entity_paragraph column
plot_most_frequent_words(df, 'entity_paragraph', top_n=20)

In [34]:
# Plot the most frequent words in the entity_paragraph column
plot_most_frequent_words(df_test, 'entity_paragraph', top_n=20)

In [35]:
def remove_stopwords(text):
    '''Removes stopwords from the text.'''
    stop_words = set(stopwords.words('english'))
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

df['entity_paragraph'] = df['entity_paragraph'].apply(remove_stopwords)
df_test['entity_paragraph'] = df_test['entity_paragraph'].apply(remove_stopwords)

In [36]:
# Plot the most frequent words in the entity_paragraph column after stopword removal
plot_most_frequent_words(df, 'entity_paragraph', top_n=20)

In [37]:
# Plot the most frequent words in the entity_paragraph column after stopword removal
plot_most_frequent_words(df_test, 'entity_paragraph', top_n=20)

In [38]:
# plot the distribution of main_role
# Create a bar plot of the fine-grained role counts
fig = px.bar(df, 
             x='main_role', 
             title='Distribution of Main Roles',
             labels={'main_role': 'Main Role'},
             template='plotly_dark'
             )  # Optional: Dark theme

# Show the plot
fig.show()

In [39]:
unique_roles = set()

for roles in df['fine_grained_roles']:
    unique_roles.update(roles)

# change unique_roles to list
unique_roles = list(unique_roles)

In [40]:
# Fine-grained roles column has roles stored as a list. plot the distribution of fine-grained roles
# Create a list to store the count of each fine-grained role
fine_grained_role_counts = []
unique_roles
# Iterate over each fine-grained role
for role in unique_roles:
    # Count the number of occurrences of the role in the fine_grained_roles column
    count = df[role].sum()
    fine_grained_role_counts.append((role, count))

# Create a DataFrame from the list of role counts
fine_grained_role_counts_df = pd.DataFrame(fine_grained_role_counts, columns=['Role', 'Count'])

# Sort the DataFrame by the count of roles in descending order
fine_grained_role_counts_df = fine_grained_role_counts_df.sort_values(by='Count', ascending=False)

# Create a bar plot of the fine-grained role counts
fig = px.bar(fine_grained_role_counts_df, 
             x='Role', 
             y='Count', 
             title='Distribution of Fine-Grained Roles',
             labels={'Role': 'Fine-Grained Role', 'Count': 'Occurrences'},
             color='Count',  # Optional: Adds color to the bars based on frequency
             template='plotly_dark')  # Optional: Dark theme

# Show the plot
fig.show()

### Train-Test Split

In [41]:
df_train, df_val = train_test_split(df, test_size=0.1, random_state=42, stratify=df['main_role'])

In [42]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [43]:
df_train.shape, df_val.shape, df_test.shape

((617, 29), (69, 29), (91, 5))

In [44]:
# plot the distribution of main_role
# Create a bar plot of the fine-grained role counts
fig = px.bar(df_train, 
             x='main_role', 
             title='Distribution of Main Roles in Train',
             labels={'main_role': 'Main Role'},
             template='plotly_dark')  # Optional: Dark theme

# Show the plot
fig.show()

In [45]:
# plot the distribution of main_role
# Create a bar plot of the fine-grained role counts
fig = px.bar(df_val, 
             x='main_role', 
             title='Distribution of Main Roles Val',
             labels={'main_role': 'Main Role'},
             template='plotly_dark')  # Optional: Dark theme

# Show the plot
fig.show()

In [46]:
df_train.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous,entity_paragraph
0,EN_UA_300053.txt,Kiev,144,147,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,top french politician claimed large part weste...
1,EN_UA_300040.txt,Ukrainians,1912,1921,Innocent,[Victim],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,one speeches american voters said want make am...
2,EN_CC_200022.txt,World Economic Forum,408,427,Antagonist,[Conspirator],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,demark become first country world introduce ca...
3,EN_CC_100035.txt,Gates,1634,1638,Antagonist,[Conspirator],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,livestock earth thousands year according bunch...
4,EN_UA_300036.txt,The United States,84,100,Protagonist,[Virtuous],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,united states plans pick slack producing eleme...


In [47]:
df_val.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous,entity_paragraph
0,EN_UA_003579.txt,international organizations,306,332,Protagonist,[Peacemaker],0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,ajor international organizations issued appeal...
1,EN_UA_029155.txt,Volodymyr Zelensky,160,177,Antagonist,"[Saboteur, Conspirator]",0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,former head german intelligence revealed ukrai...
2,EN_UA_003579.txt,IMF,846,848,Protagonist,[Peacemaker],0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,critical quickly provide support food insecure...
3,EN_UA_300082.txt,Kiev,975,978,Antagonist,[Terrorist],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,unfortunately oleg tsarev far first victim naz...
4,EN_UA_013710.txt,ICC,207,209,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,collective west hypocrisy rules narrative rece...


In [48]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...


## TF-IDF + Label PowerSet

In [49]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(df_train["entity_paragraph"])
X_val_vec = vectorizer.transform(df_val["entity_paragraph"])
X_test_vec = vectorizer.transform(df_test["entity_paragraph"])

In [50]:
df_train.columns

Index(['article_id', 'entity_mention', 'start_offset', 'end_offset',
       'main_role', 'fine_grained_roles', 'Bigot', 'Conspirator', 'Corrupt',
       'Deceiver', 'Exploited', 'Foreign Adversary', 'Forgotten', 'Guardian',
       'Incompetent', 'Instigator', 'Martyr', 'Peacemaker', 'Rebel',
       'Saboteur', 'Scapegoat', 'Spy', 'Terrorist', 'Traitor', 'Tyrant',
       'Underdog', 'Victim', 'Virtuous', 'entity_paragraph'],
      dtype='object')

In [51]:
y_train = df_train.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)
y_val = df_val.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)

In [735]:
Models_acc = {}
Models_micro = {}
Models_macro = {}
Models_emr = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    if modelName == "Power Set SVC":
        exact_matches = np.sum(np.all(np.array(test_labels) == np.array(predictions.todense()), axis=1))  # Compare if all labels match for each sample
        emr = exact_matches / len(y_val)
    else:
        exact_matches = np.sum(np.all(np.array(test_labels) == np.array(predictions), axis=1))  # Compare if all labels match for each sample
        emr = exact_matches / len(y_val)

    hamLoss = hamming_loss(test_labels, predictions)
    
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nExact Match Ratio: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, emr, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    
    Models_acc[modelName] = accuracy
    Models_micro[modelName] = micro_f1
    Models_macro[modelName] = macro_f1
    Models_emr[modelName] = {emr}

In [736]:
knn = KNeighborsClassifier()

knn.fit(X_train_vec, y_train)
knnPredictions = knn.predict(X_val_vec)
metricsReport("knn", y_val, knnPredictions)

------knn Model Metrics-----
Accuracy: 0.1159
Hamming Loss: 0.0468
Exact Match Ratio: 0.1159
Precision:
  - Macro: 0.1727
  - Micro: 0.5625
Recall:
  - Macro: 0.1136
  - Micro: 0.1233
F1-measure:
  - Macro: 0.1328
  - Micro: 0.2022


In [737]:
dt = DecisionTreeClassifier()
dt.fit(X_train_vec, y_train)
dtPreds = dt.predict(X_val_vec)
metricsReport("Decision Tree", y_val, dtPreds)

------Decision Tree Model Metrics-----
Accuracy: 0.2319
Hamming Loss: 0.0672
Exact Match Ratio: 0.2319
Precision:
  - Macro: 0.1581
  - Micro: 0.2899
Recall:
  - Macro: 0.1899
  - Micro: 0.2740
F1-measure:
  - Macro: 0.1626
  - Micro: 0.2817


In [738]:
bag = OneVsRestClassifier(BaggingClassifier(n_jobs=-1))
bag.fit(X_train_vec, y_train)
bagPreds = bag.predict(X_val_vec)
metricsReport("Bagging", y_val, bagPreds)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

------Bagging Model Metrics-----
Accuracy: 0.1739
Hamming Loss: 0.0474
Exact Match Ratio: 0.1739
Precision:
  - Macro: 0.2336
  - Micro: 0.5161
Recall:
  - Macro: 0.1342
  - Micro: 0.2192
F1-measure:
  - Macro: 0.1625
  - Micro: 0.3077


In [739]:
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train_vec, y_train)
rfPreds = rf.predict(X_val_vec)
metricsReport("Random Forest", y_val, rfPreds)

------Random Forest Model Metrics-----
Accuracy: 0.2029
Hamming Loss: 0.0455
Exact Match Ratio: 0.2029
Precision:
  - Macro: 0.2602
  - Micro: 0.5667
Recall:
  - Macro: 0.1596
  - Micro: 0.2329
F1-measure:
  - Macro: 0.1857
  - Micro: 0.3301


In [740]:
boostClassifier = OneVsRestClassifier(GradientBoostingClassifier())
boostClassifier.fit(X_train_vec, y_train)
boostPreds = boostClassifier.predict(X_val_vec)
metricsReport("Boosting", y_val, boostPreds)

------Boosting Model Metrics-----
Accuracy: 0.2029
Hamming Loss: 0.0487
Exact Match Ratio: 0.2029
Precision:
  - Macro: 0.2913
  - Micro: 0.4878
Recall:
  - Macro: 0.1839
  - Micro: 0.2740
F1-measure:
  - Macro: 0.2080
  - Micro: 0.3509


In [741]:
nb = OneVsRestClassifier(MultinomialNB())
nb.fit(X_train_vec, y_train)

nbPreds = nb.predict(X_val_vec)
metricsReport("Multinomial NB", y_val, nbPreds)

------Multinomial NB Model Metrics-----
Accuracy: 0.0145
Hamming Loss: 0.0487
Exact Match Ratio: 0.0145
Precision:
  - Macro: 0.0227
  - Micro: 0.3333
Recall:
  - Macro: 0.0038
  - Micro: 0.0137
F1-measure:
  - Macro: 0.0065
  - Micro: 0.0263


In [742]:
svm = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svm.fit(X_train_vec, y_train)

svmPreds = svm.predict(X_val_vec)
metricsReport("SVC Sq. Hinge Loss", y_val, svmPreds)

------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.2174
Hamming Loss: 0.0448
Exact Match Ratio: 0.2174
Precision:
  - Macro: 0.2648
  - Micro: 0.5806
Recall:
  - Macro: 0.1672
  - Micro: 0.2466
F1-measure:
  - Macro: 0.1923
  - Micro: 0.3462


In [743]:
powerSetSVC = LabelPowerset(LinearSVC())
powerSetSVC.fit(X_train_vec, y_train)

powerSetSVCPreds = powerSetSVC.predict(X_val_vec)
metricsReport("Power Set SVC", y_val, powerSetSVCPreds)

------Power Set SVC Model Metrics-----
Accuracy: 0.3478
Hamming Loss: 0.0599
Exact Match Ratio: 0.3478
Precision:
  - Macro: 0.2098
  - Micro: 0.3750
Recall:
  - Macro: 0.2574
  - Micro: 0.3699
F1-measure:
  - Macro: 0.2250
  - Micro: 0.3724


In [744]:
print("  Model Name " + " "*10 + "| Exact Match Ratio Score")
print("-------------------------------------------")
for key, value in Models_emr.items():
    print("  " + key, " "*(20-len(key)) + "|", value)
    print("-------------------------------------------")

  Model Name           | Exact Match Ratio Score
-------------------------------------------
  knn                  | {0.11594202898550725}
-------------------------------------------
  Decision Tree        | {0.2318840579710145}
-------------------------------------------
  Bagging              | {0.17391304347826086}
-------------------------------------------
  Random Forest        | {0.2028985507246377}
-------------------------------------------
  Boosting             | {0.2028985507246377}
-------------------------------------------
  Multinomial NB       | {0.014492753623188406}
-------------------------------------------
  SVC Sq. Hinge Loss   | {0.21739130434782608}
-------------------------------------------
  Power Set SVC        | {0.34782608695652173}
-------------------------------------------


In [344]:
# Convert powerSetSVCPreds to a DataFrame with the same column names as y_val
powerSetSVCPreds_df = pd.DataFrame(np.array(powerSetSVCPreds.todense()), columns=y_val.columns)

In [345]:
powerSetSVCPreds_df.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [346]:
powerSetSVCPreds_test = powerSetSVC.predict(X_test_vec)

In [347]:
# Convert powerSetSVCPreds to a DataFrame with the same column names as y_val
powerSetSVCPreds_df_test = pd.DataFrame(np.array(powerSetSVCPreds_test.todense()), columns=y_val.columns)

In [348]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...


In [349]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...


In [350]:
powerSetSVCPreds_df_test.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [351]:
# Function to convert each row into a comma-separated string of column names where the value is 1
def get_column_names_with_ones(row):
    return ','.join([col for col in row.index if row[col] == 1])

# Apply the function to each row of the DataFrame
powerSetSVCPreds_df_test_single_column = powerSetSVCPreds_df_test.apply(get_column_names_with_ones, axis=1)

# Convert the result to a DataFrame with a single column
powerSetSVCPreds_df_test_single_column = pd.DataFrame(powerSetSVCPreds_df_test_single_column, columns=['Predictions'])

In [352]:
powerSetSVCPreds_df_test_single_column.head()

,Predictions
0,Incompetent
1,Victim
2,Corrupt
3,Conspirator
4,Deceiver


In [353]:
# Rename the Predictions column to the desired name, e.g., 'Labels'
powerSetSVCPreds_df_test_single_column.rename(columns={'Predictions': 'fine_grained_roles'}, inplace=True)

# Print the result to verify
powerSetSVCPreds_df_test_single_column.head()

,fine_grained_roles
0,Incompetent
1,Victim
2,Corrupt
3,Conspirator
4,Deceiver


In [354]:
# Define the role lists
PROTAGONISTS = ['Guardian', 'Martyr', 'Peacemaker', 'Rebel', 'Underdog', 'Virtuous']
ANTAGONISTS = ['Instigator', 'Conspirator', 'Tyrant', 'Foreign Adversary', 'Traitor', 'Spy', 'Saboteur', 'Corrupt', 'Incompetent', 'Terrorist', 'Deceiver', 'Bigot']
INNOCENTS = ['Forgotten', 'Exploited', 'Victim', 'Scapegoat']

# Initialize an empty list to store indices of ambiguous rows
ambiguous = []

# Function to decide the main role based on the labels
def decide_main_role(labels):
    # Split the comma-separated labels
    roles = labels.split(',')

    # Create counters for each category
    protagonist_count = sum(1 for role in roles if role in PROTAGONISTS)
    antagonist_count = sum(1 for role in roles if role in ANTAGONISTS)
    innocent_count = sum(1 for role in roles if role in INNOCENTS)

    # If more than one category is represented, check for ambiguity
    if (protagonist_count > 0 and antagonist_count > 0) or \
       (protagonist_count > 0 and innocent_count > 0) or \
       (antagonist_count > 0 and innocent_count > 0):
        ambiguous.append(labels)  # Add to ambiguous list
        return None  # Set main_role as None for ambiguous cases
    
    # Majority voting - select the category with the highest count
    if protagonist_count > max(antagonist_count, innocent_count):
        return 'Protagonist'
    elif antagonist_count > max(protagonist_count, innocent_count):
        return 'Antagonist'
    elif innocent_count > max(protagonist_count, antagonist_count):
        return 'Innocent'
    else:
        # Handle case where there's no majority (all are 0)
        return None

# Apply the function to create the 'main_role' column
powerSetSVCPreds_df_test_single_column['main_role'] = powerSetSVCPreds_df_test_single_column['fine_grained_roles'].apply(decide_main_role)

# Print the list of ambiguous rows (optional)
print("Ambiguous rows:", ambiguous)

powerSetSVCPreds_df_test_single_column.head()

Ambiguous rows: []


,fine_grained_roles,main_role
0,Incompetent,Antagonist
1,Victim,Innocent
2,Corrupt,Antagonist
3,Conspirator,Antagonist
4,Deceiver,Antagonist


In [355]:
# Concatenate df_test and powerSetSVCPreds_df_test along the columns (axis=1)
combined_df = pd.concat([df_test, powerSetSVCPreds_df_test_single_column], axis=1)

# Print the result to verify
combined_df.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph,fine_grained_roles,main_role
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...,Incompetent,Antagonist
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...,Victim,Innocent
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...,Corrupt,Antagonist
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...,Conspirator,Antagonist
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...,Deceiver,Antagonist


In [356]:
df_dev = combined_df[['article_id', 'entity_mention', 'start_offset', 'end_offset', 
                      'main_role', 'fine_grained_roles']]

In [357]:
df_dev.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles
0,EN_UA_DEV_100012.txt,Washington,1441,1450,Antagonist,Incompetent
1,EN_UA_DEV_100012.txt,Americans,2397,2405,Innocent,Victim
2,EN_CC_200053.txt,Rishi Sunak,85,95,Antagonist,Corrupt
3,EN_CC_200040.txt,Extinction Rebellion,189,208,Antagonist,Conspirator
4,EN_CC_200070.txt,Eric Garcetti,89,101,Antagonist,Deceiver


In [358]:
# Open the file in write mode
with open('output_file_final_tfidf_lp.txt', 'w', encoding='utf-8') as f:
    # Write the header
    header = df_dev.columns.tolist()
    f.write('\t'.join(header) + '\n')
    
    # Iterate through each row and write the data
    for _, row in df_dev.iterrows():
        # Replace commas in fine_grained_roles with tabs
        fine_grained_roles = row['fine_grained_roles'].replace(',', '\t')
        
        # Create a list of row values
        row_values = [str(row[col]) if col != 'fine_grained_roles' else fine_grained_roles for col in df_dev.columns]
        
        # Write the row to the file, separating values with tabs
        f.write('\t'.join(row_values) + '\n')

# Print to verify
print("DataFrame saved successfully as a tab-separated file.")

DataFrame saved successfully as a tab-separated file.


## LLM

In [745]:
df_train.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous,entity_paragraph
0,EN_UA_300053.txt,Kiev,144,147,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,top french politician claimed large part weste...
1,EN_UA_300040.txt,Ukrainians,1912,1921,Innocent,[Victim],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,one speeches american voters said want make am...
2,EN_CC_200022.txt,World Economic Forum,408,427,Antagonist,[Conspirator],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,demark become first country world introduce ca...
3,EN_CC_100035.txt,Gates,1634,1638,Antagonist,[Conspirator],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,livestock earth thousands year according bunch...
4,EN_UA_300036.txt,The United States,84,100,Protagonist,[Virtuous],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,united states plans pick slack producing eleme...


In [746]:
df_val.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous,entity_paragraph
0,EN_UA_003579.txt,international organizations,306,332,Protagonist,[Peacemaker],0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,ajor international organizations issued appeal...
1,EN_UA_029155.txt,Volodymyr Zelensky,160,177,Antagonist,"[Saboteur, Conspirator]",0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,former head german intelligence revealed ukrai...
2,EN_UA_003579.txt,IMF,846,848,Protagonist,[Peacemaker],0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,critical quickly provide support food insecure...
3,EN_UA_300082.txt,Kiev,975,978,Antagonist,[Terrorist],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,unfortunately oleg tsarev far first victim naz...
4,EN_UA_013710.txt,ICC,207,209,Antagonist,[Corrupt],0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,collective west hypocrisy rules narrative rece...


In [747]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...


In [748]:
# get the unique values of fine_grained_roles
unique_roles = set()

for roles in df['fine_grained_roles']:
    unique_roles.update(roles)
# change unique_roles to list
unique_roles = list(unique_roles)
unique_roles

['Scapegoat',
 'Conspirator',
 'Tyrant',
 'Foreign Adversary',
 'Terrorist',
 'Spy',
 'Bigot',
 'Exploited',
 'Traitor',
 'Guardian',
 'Saboteur',
 'Peacemaker',
 'Instigator',
 'Forgotten',
 'Deceiver',
 'Incompetent',
 'Victim',
 'Rebel',
 'Virtuous',
 'Martyr',
 'Corrupt',
 'Underdog']

In [749]:
from datasets import Dataset, DatasetDict

# Convert the pandas Series (list of dictionaries) into a Dataset
train_dataset = Dataset.from_pandas(df_train)

# Similarly, you can convert validation and test data if needed
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)  # If test data exists

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    # 'test': test_dataset  # Uncomment if test dataset exists
})

test_dict = DatasetDict({
    'test': test_dataset
})

# Now dataset_dict is a DatasetDict
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited', 'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent', 'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat', 'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim', 'Virtuous', 'entity_paragraph'],
        num_rows: 617
    })
    validation: Dataset({
        features: ['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited', 'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent', 'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat', 'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim', 'Virtuous', 'entity_paragraph'],
        num_rows: 69
    })
})

In [750]:
example = test_dict['test'][0]
example

{'article_id': 'EN_UA_DEV_100012.txt',
 'entity_mention': 'Washington',
 'start_offset': 1441,
 'end_offset': 1450,
 'entity_paragraph': 'washington cannot afford allow russia achieve victory ukraine conflict would mean losing direct access vast mineral assets view us senator lindsey graham interview face nation cbs news june'}

In [751]:
labels = [label for label in dataset_dict['train'].features.keys() 
          if label not in ['entity_paragraph', 'article_id', 'entity_mention','start_offset', 
                           'end_offset', 'main_role', 'fine_grained_roles']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Bigot',
 'Conspirator',
 'Corrupt',
 'Deceiver',
 'Exploited',
 'Foreign Adversary',
 'Forgotten',
 'Guardian',
 'Incompetent',
 'Instigator',
 'Martyr',
 'Peacemaker',
 'Rebel',
 'Saboteur',
 'Scapegoat',
 'Spy',
 'Terrorist',
 'Traitor',
 'Tyrant',
 'Underdog',
 'Victim',
 'Virtuous']

In [752]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["entity_paragraph"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

def preprocess_data_test(examples):
  # take a batch of texts
  text = examples["entity_paragraph"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    
  return encoding

In [753]:
encoded_dataset = dataset_dict.map(preprocess_data, batched=True, 
                                   remove_columns=dataset_dict['train'].column_names)
encoded_test_dataset = test_dict.map(preprocess_data_test, batched=True, 
                                   remove_columns=test_dict['test'].column_names)

Map:   0%|          | 0/617 [00:00<?, ? examples/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [754]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [755]:
tokenizer.decode(example['input_ids'])

'[CLS] top french politician claimed large part western aid kiev embezzled ukrainian officials despite president zelensky assurances used fight russia [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [756]:
example['labels']

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [757]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Corrupt']

In [758]:
encoded_dataset.set_format("torch")

In [759]:
encoded_test_dataset.set_format("torch")

In [760]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [761]:
batch_size = 8
metric_name = "exact_match"

In [762]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none"
    #push_to_hub=True,
)

In [763]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    exact_match = np.mean(np.all(y_pred == y_true, axis=1))
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'exact_match': exact_match}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [764]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [765]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2327,  2413,  3761,  3555,  2312,  2112,  2530,  4681, 12100,
         7861,  4783, 17269,  5969,  4584,  2750,  2343, 27838,  7770,  5874,
        16375,  2015,  2109,  2954,  3607,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [766]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6640, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.4867, -0.0868,  0.0936,  0.0736,  0.1265, -0.0356,  0.0250, -0.1474,
         -0.0979, -0.1906,  0.1034,  0.0752,  0.1218,  0.0356,  0.1256, -0.2577,
          0.1056, -0.4471, -0.5154,  0.1015, -0.0490, -0.0031]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [767]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [768]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Exact Match
1,No log,0.355427,0.000000,0.500000,0.000000,0.000000
2,No log,0.259690,0.000000,0.500000,0.000000,0.000000
3,No log,0.229618,0.000000,0.500000,0.000000,0.000000
4,No log,0.218181,0.000000,0.500000,0.000000,0.000000
5,No log,0.215035,0.000000,0.500000,0.000000,0.000000


TrainOutput(global_step=195, training_loss=0.30127311119666467, metrics={'train_runtime': 82.2287, 'train_samples_per_second': 37.517, 'train_steps_per_second': 2.371, 'total_flos': 202960840972800.0, 'train_loss': 0.30127311119666467, 'epoch': 5.0})

In [769]:
trainer.evaluate()

{'eval_loss': 0.3554273545742035,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_exact_match': 0.0,
 'eval_runtime': 0.5113,
 'eval_samples_per_second': 134.956,
 'eval_steps_per_second': 9.779,
 'epoch': 5.0}

## BERT+CNN

In [382]:
import torch
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer, T5EncoderModel, T5Tokenizer
from sklearn.model_selection import train_test_split
import re
import os
import requests
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import GPT2Tokenizer, GPT2Model
from transformers import T5Tokenizer, T5EncoderModel
from transformers import XLNetTokenizer, XLNetModel

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [383]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# If CUDA is available, we will fetch the name of the CUDA device, otherwise we will return a message stating CUDA is not available.
cuda_device_info = torch.cuda.get_device_name(0) if cuda_available else "CUDA is not available."

cuda_available, cuda_device_info

(True, 'Tesla T4')

In [384]:
# # Calculate the number of words in each row of the entity_paragraph column
# df_train['word_count'] = df_train['entity_paragraph'].apply(lambda x: len(str(x).split()))

# # Find the maximum word count
# max_word_count = df_train['word_count'].max()

# # Get the row with the maximum word count
# row_with_max_words = df_train.loc[df_train['word_count'].idxmax()]

# print(f"The highest number of words in a row is: {max_word_count}")
# print("Row with the highest word count:")
# print(row_with_max_words)

In [385]:
class LM_EMBED:

    def __init__(self, language_model, max_len):
        self.lang_model = language_model
        self.max_len = max_len

        # Import tokenizer and model based on the specified language model
        if self.lang_model == 'BERT':
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            self.model = BertModel.from_pretrained('bert-base-uncased')
        elif self.lang_model == 'ROBERTA':
            self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
            self.model = RobertaModel.from_pretrained('roberta-base')
        elif self.lang_model == 'GPT2':
            self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.model = GPT2Model.from_pretrained('gpt2')
            # Assign a padding token if not already present
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.config.pad_token_id = self.tokenizer.pad_token_id
        elif self.lang_model == 'T5':
            self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
            self.model = T5EncoderModel.from_pretrained('t5-base')
        elif self.lang_model == 'XLNET':
            self.tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
            self.model = XLNetModel.from_pretrained('xlnet-base-cased')
        else:
            raise ValueError(f"Unsupported language model: {self.lang_model}")

    def extract_word_embs(self, text_df, filename):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")

        self.model = self.model.to(device)
        self.model.eval()

        texts = text_df['entity_paragraph'].tolist()

        # Tokenize and encode the texts
        tokens = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True,
            return_tensors='pt'
        )
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)

        torch.cuda.empty_cache()

        for i in range(0, len(input_ids), 10):
            if i % 100 == 0:
                print(f"Processing batch ending at {i}")
            with torch.no_grad():
                embeddings = self.model(input_ids=input_ids[i:i+10],
                                        attention_mask=attention_mask[i:i+10])[0]
                emb_array = embeddings.cpu().numpy()

                if i == 0:
                    embedding_res = emb_array
                else:
                    embedding_res = np.concatenate((embedding_res, emb_array))

        features = self.extract_features(embedding_res, attention_mask)
        padded_arr = self.pad(features)

        print("Saving embeddings...")
        np.save(filename, padded_arr)

    def extract_features(self, emb_res, att_msk):
        features = []
        for seq_num in range(len(emb_res)):
            seq_len = (att_msk[seq_num] == 1).sum()

            if self.lang_model in ['BERT', 'ROBERTA']:
                seq_emb = emb_res[seq_num][1:seq_len-1]
            elif self.lang_model == 'T5':
                seq_emb = emb_res[seq_num][:seq_len-1]
            elif self.lang_model == 'XLNET':
                padded_seq_len = len(att_msk[seq_num])
                seq_emb = emb_res[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
            elif self.lang_model == 'GPT2':
                seq_emb = emb_res[seq_num][:seq_len]
            else:
                raise ValueError(f"Unsupported language model: {self.lang_model}")

            features.append(seq_emb)
        return np.array(features, dtype=object)

    def pad(self, features):
        dim1 = self.max_len
        dim2 = features[0].shape[1]

        for i, feature in enumerate(features):
            if i % 100 == 0:
                print(f"Padding batch {i}")

            padded_feature = np.zeros((dim1, dim2))
            padded_feature[:feature.shape[0], :feature.shape[1]] = feature

            if i == 0:
                padded_arr = padded_feature
            elif i == 1:
                padded_arr = np.stack((padded_arr, padded_feature), axis=0)
            else:
                reshaped_feature = padded_feature.reshape(1, dim1, dim2)
                padded_arr = np.vstack((padded_arr, reshaped_feature))

        return padded_arr

In [386]:
# Specifying the max sequence length in the given dataset (255 for the LMPred Dataset), 
# then adding 2 to account for special [CLS, SEP] tokens added by the language models):
max_seq_len = 104

BERT = LM_EMBED('BERT', max_seq_len)
ROBERTA = LM_EMBED('ROBERTA', max_seq_len)
GPT2 = LM_EMBED('GPT2', max_seq_len)
T5 = LM_EMBED('T5', max_seq_len)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [387]:
X_train = df_train["entity_paragraph"]
X_val = df_val["entity_paragraph"]
X_test = df_test["entity_paragraph"]

In [232]:
BERT.extract_word_embs(df_train, "/kaggle/working/BERT_EMBED_X_TRAIN.npy")
BERT.extract_word_embs(df_val, "/kaggle/working/BERT_EMBED_X_VAL.npy")
BERT.extract_word_embs(df_test, "/kaggle/working/BERT_EMBED_X_TEST.npy")

Using device: cuda:0
Processing batch ending at 0
Processing batch ending at 100
Processing batch ending at 200
Processing batch ending at 300
Processing batch ending at 400
Processing batch ending at 500
Processing batch ending at 600
Padding batch 0
Padding batch 100
Padding batch 200
Padding batch 300
Padding batch 400
Padding batch 500
Padding batch 600
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...


In [239]:
ROBERTA.extract_word_embs(df_train, "/kaggle/working/ROBERTA_EMBED_X_TRAIN.npy")
ROBERTA.extract_word_embs(df_val, "/kaggle/working/ROBERTA_EMBED_X_VAL.npy")
ROBERTA.extract_word_embs(df_test, "/kaggle/working/ROBERTA_EMBED_X_TEST.npy")

Using device: cuda:0
Processing batch ending at 0
Processing batch ending at 100
Processing batch ending at 200
Processing batch ending at 300
Processing batch ending at 400
Processing batch ending at 500
Processing batch ending at 600
Padding batch 0
Padding batch 100
Padding batch 200
Padding batch 300
Padding batch 400
Padding batch 500
Padding batch 600
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...


In [247]:
T5.extract_word_embs(df_train, "/kaggle/working/T5_EMBED_X_TRAIN.npy")
T5.extract_word_embs(df_val, "/kaggle/working/T5_EMBED_X_VAL.npy")
T5.extract_word_embs(df_test, "/kaggle/working/T5_EMBED_X_TEST.npy")

Using device: cuda:0
Processing batch ending at 0
Processing batch ending at 100
Processing batch ending at 200
Processing batch ending at 300
Processing batch ending at 400
Processing batch ending at 500
Processing batch ending at 600
Padding batch 0
Padding batch 100
Padding batch 200
Padding batch 300
Padding batch 400
Padding batch 500
Padding batch 600
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...
Using device: cuda:0
Processing batch ending at 0
Padding batch 0
Saving embeddings...


In [390]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, InputLayer, Conv2D, MaxPooling2D, BatchNormalization, Input
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.ticker as mtick
from matplotlib import pyplot as plt

In [391]:
y_train.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [392]:
y_val.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [625]:
def load_X_data(lang_model):
  if lang_model =='BERT':
    X_train = np.load('/kaggle/working/BERT_EMBED_X_TRAIN.npy')
    X_val = np.load('/kaggle/working/BERT_EMBED_X_VAL.npy')
  elif lang_model =='ROBERTA':
    X_train = np.load('/kaggle/working/ROBERTA_EMBED_X_TRAIN.npy')
    X_val = np.load('/kaggle/working/ROBERTA_EMBED_X_VAL.npy')
  elif lang_model =='T5':
    X_train = np.load('/kaggle/working/T5_EMBED_X_TRAIN.npy')
    X_val = np.load('/kaggle/working/T5_EMBED_X_VAL.npy')
  elif lang_model =='GPT2':
    X_train = np.load('/kaggle/working/T5XL_BFD_EMBED_X_TRAIN.npy')
    X_val = np.load('/kaggle/working/T5XL_BFD_EMBED_X_VAL.npy')
  elif lang_model =='XLNET':
    X_train = np.load('/kaggle/working/XLNET_EMMBED_X_TRAIN.npy')
    X_val = np.load('/kaggle/working/XLNET_EMMBED_X_VAL.npy')

  return X_train, X_val

In [504]:
X_train, X_val = load_X_data('T5')

In [505]:
X_train.shape, X_val.shape

((617, 104, 768), (69, 104, 768))

In [506]:
type(X_train)

numpy.ndarray

In [507]:
type(y_train)

numpy.ndarray

In [508]:
# Define the Dataset class
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels=None):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        if self.labels is not None:
            return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)
        else:
            return torch.tensor(self.embeddings[idx], dtype=torch.float32)

class CNNClassifier(nn.Module):
    def __init__(self, input_channels, sequence_length, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(128 * (sequence_length // 2), num_classes)  # Adjust dimensions after pooling

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change to (batch_size, input_channels, sequence_length)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x  # No sigmoid here

In [509]:
# Calculate class weights
label_sums = np.sum(y_train, axis=0)  # Sum occurrences of each class
weight_per_class = 1.0 / (label_sums + 1e-6)  # Avoid division by zero

# Convert to a PyTorch tensor
class_weights = torch.tensor(weight_per_class, dtype=torch.float32).to(device)

# Use BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)  # Use pos_weight for class balancing

In [510]:
y_train = df_train.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)
y_val = df_val.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)

y_train = y_train.values          # Convert DataFrame to numpy
y_val = y_val.values              # Convert DataFrame to numpy

In [511]:
train_dataset = EmbeddingDataset(X_train, y_train)
val_dataset = EmbeddingDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize Model, Loss, and Optimizer
input_channels = X_train.shape[2]
sequence_length = X_train.shape[1]
num_classes = y_train.shape[1]

model = CNNClassifier(input_channels=input_channels, sequence_length=sequence_length, num_classes=num_classes)
# class_weights = torch.tensor([weight_per_class], dtype=torch.float32).to(device)
# criterion = nn.BCELoss(weight=class_weights)
# criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multilabel classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [512]:
# Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(10): # Number of epochs
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0.0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            y_true.append(y_batch.cpu().numpy())
            y_pred.append((outputs.cpu().numpy() > 0.5).astype(int))

    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)

    # Metrics
    exact_matches = np.sum(np.all(np.array(y_true) == np.array(y_pred), axis=1))  # Compare if all labels match for each sample
    emr = exact_matches / len(y_val)
    f1 = f1_score(y_true, y_pred, average='micro')

    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Exact Match: {emr}, F1 Score: {f1}")

Epoch 1, Train Loss: 0.1052441205829382, Val Loss: 0.06352711468935013, Exact Match: 0.0, F1 Score: 0.0
Epoch 2, Train Loss: 0.043727355729788545, Val Loss: 0.04606101910273234, Exact Match: 0.0, F1 Score: 0.0
Epoch 3, Train Loss: 0.023998790327459572, Val Loss: 0.020447660547991593, Exact Match: 0.0, F1 Score: 0.0
Epoch 4, Train Loss: 0.01380921145901084, Val Loss: 0.016613183543086052, Exact Match: 0.0, F1 Score: 0.0
Epoch 5, Train Loss: 0.012229485670104623, Val Loss: 0.015889011633892853, Exact Match: 0.0, F1 Score: 0.0
Epoch 6, Train Loss: 0.01131794941611588, Val Loss: 0.015737893680731457, Exact Match: 0.0, F1 Score: 0.0
Epoch 7, Train Loss: 0.010819756891578436, Val Loss: 0.016229749967654545, Exact Match: 0.0, F1 Score: 0.0
Epoch 8, Train Loss: 0.010554645396769046, Val Loss: 0.015899337207277615, Exact Match: 0.0, F1 Score: 0.0
Epoch 9, Train Loss: 0.010238636238500475, Val Loss: 0.016197705330948036, Exact Match: 0.0, F1 Score: 0.0
Epoch 10, Train Loss: 0.009925945149734616,

In [626]:
def load_X_Test_data(lang_model):
  if lang_model =='BERT':
    print("bert")
    X_test = np.load('/kaggle/working/BERT_EMBED_X_TEST.npy')
  elif lang_model =='ROBERTA':
    print("here")
    X_test = np.load('/kaggle/working/ROBERTA_EMBED_X_TEST.npy')
  elif lang_model =='T5':
    print("here")
    X_test = np.load('/kaggle/working/T5_EMBED_X_TEST.npy')
  elif lang_model =='GPT2':
    X_test = np.load('Embeddings/T5XL_UNI/T5_VELTRI_X_TEST_INDEP.npy')
  elif lang_model =='T5XL_BFD':
    print("T5XL_BFD")
    X_test = np.load('/kaggle/working/T5XL_BFD_EMBED_X_TEST.npy')
  elif lang_model =='XLNET':
    print("here")
    X_test = np.load('/kaggle/working/XLNET_EMMBED_X_TEST.npy')
  elif lang_model =='VELTRI':
    X_test = np.load('Datasets/VELTRI/X_test_VELTRI_Conv.npy')
  
  
  return X_test

In [522]:
X_test = load_X_Test_data('T5')

here


In [523]:
columns = df_train.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset',
       'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)

columns.columns

Index(['Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous'],
      dtype='object')

In [524]:
test_dataset = EmbeddingDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
test_preds = []

with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        test_preds.append((outputs.cpu().numpy() > 0.5).astype(int))

test_preds = np.vstack(test_preds)
y_pred = pd.DataFrame(test_preds, columns=columns.columns)

y_pred.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [525]:
# Function to convert each row into a comma-separated string of column names where the value is 1
def get_column_names_with_ones(row):
    return ','.join([col for col in row.index if row[col] == 1])

# Apply the function to each row of the DataFrame
y_pred_new = y_pred.apply(get_column_names_with_ones, axis=1)

# Convert the result to a DataFrame with a single column
y_pred_new = pd.DataFrame(y_pred_new, columns=['Predictions'])
y_pred_new.head()

,Predictions
0,
1,
2,
3,
4,


## BERT+LABELPOWERSET

In [817]:
X_train, X_val = load_X_data('T5')
X_test = load_X_Test_data('T5')

here


In [818]:
X_train.shape

(617, 104, 768)

In [819]:
def average_pool_embeddings(embeddings):
    """
    Applies average pooling to reduce the shape of embeddings from (N, L, D) to (N, D).
    
    Parameters:
        embeddings (numpy.ndarray): The input tensor of shape (N, L, D), where
                                     N is the number of samples,
                                     L is the max sequence length,
                                     D is the embedding dimension.
    
    Returns:
        numpy.ndarray: The output tensor of shape (N, D).
    """
    if len(embeddings.shape) != 3:
        raise ValueError("Input embeddings must have a shape of (N, L, D).")
    
    # Average pooling along the sequence length axis
    pooled_embeddings = np.mean(embeddings, axis=1)
    return pooled_embeddings

X_train = average_pool_embeddings(X_train)
X_val = average_pool_embeddings(X_val)
X_test = average_pool_embeddings(X_test)

In [820]:
X_train.shape, X_val.shape, X_test.shape

((617, 768), (69, 768), (91, 768))

In [821]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
knnPredictions = knn.predict(X_val)
metricsReport("knn", y_val, knnPredictions)

------knn Model Metrics-----
Accuracy: 0.1014
Hamming Loss: 0.0461
Exact Match Ratio: 0.1014
Precision:
  - Macro: 0.1212
  - Micro: 0.6154
Recall:
  - Macro: 0.0530
  - Micro: 0.1096
F1-measure:
  - Macro: 0.0725
  - Micro: 0.1860


In [822]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dtPreds = dt.predict(X_val)
metricsReport("Decision Tree", y_val, dtPreds)

------Decision Tree Model Metrics-----
Accuracy: 0.2464
Hamming Loss: 0.0659
Exact Match Ratio: 0.2464
Precision:
  - Macro: 0.2082
  - Micro: 0.2985
Recall:
  - Macro: 0.1801
  - Micro: 0.2740
F1-measure:
  - Macro: 0.1846
  - Micro: 0.2857


In [823]:
bag = OneVsRestClassifier(BaggingClassifier(n_jobs=-1))
bag.fit(X_train, y_train)
bagPreds = bag.predict(X_val)
metricsReport("Bagging", y_val, bagPreds)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

------Bagging Model Metrics-----
Accuracy: 0.1884
Hamming Loss: 0.0468
Exact Match Ratio: 0.1884
Precision:
  - Macro: 0.2958
  - Micro: 0.5333
Recall:
  - Macro: 0.1584
  - Micro: 0.2192
F1-measure:
  - Macro: 0.1953
  - Micro: 0.3107


In [824]:
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train, y_train)
rfPreds = rf.predict(X_val)
metricsReport("Random Forest", y_val, rfPreds)

------Random Forest Model Metrics-----
Accuracy: 0.2029
Hamming Loss: 0.0461
Exact Match Ratio: 0.2029
Precision:
  - Macro: 0.2602
  - Micro: 0.5484
Recall:
  - Macro: 0.1596
  - Micro: 0.2329
F1-measure:
  - Macro: 0.1857
  - Micro: 0.3269


In [825]:
boostClassifier = OneVsRestClassifier(GradientBoostingClassifier())
boostClassifier.fit(X_train, y_train)
boostPreds = boostClassifier.predict(X_val)
metricsReport("Boosting", y_val, boostPreds)

------Boosting Model Metrics-----
Accuracy: 0.2029
Hamming Loss: 0.0468
Exact Match Ratio: 0.2029
Precision:
  - Macro: 0.2284
  - Micro: 0.5294
Recall:
  - Macro: 0.1672
  - Micro: 0.2466
F1-measure:
  - Macro: 0.1875
  - Micro: 0.3364


In [826]:
svm = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svm.fit(X_train, y_train)

svmPreds = svm.predict(X_val)
metricsReport("SVC Sq. Hinge Loss", y_val, svmPreds)

------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.0580
Hamming Loss: 0.0507
Exact Match Ratio: 0.0580
Precision:
  - Macro: 0.1136
  - Micro: 0.3333
Recall:
  - Macro: 0.0292
  - Micro: 0.0548
F1-measure:
  - Macro: 0.0455
  - Micro: 0.0941


In [827]:
powerSetSVC = LabelPowerset(LinearSVC())
powerSetSVC.fit(X_train, y_train)

powerSetSVCPreds = powerSetSVC.predict(X_val)
metricsReport("Power Set SVC", y_val, powerSetSVCPreds)

------Power Set SVC Model Metrics-----
Accuracy: 0.3478
Hamming Loss: 0.0586
Exact Match Ratio: 0.3478
Precision:
  - Macro: 0.3360
  - Micro: 0.3889
Recall:
  - Macro: 0.3202
  - Micro: 0.3836
F1-measure:
  - Macro: 0.3122
  - Micro: 0.3862


In [828]:
print("  Model Name " + " "*10 + "| Exact Match Ratio Score")
print("-------------------------------------------")
for key, value in Models_emr.items():
    print("  " + key, " "*(20-len(key)) + "|", value)
    print("-------------------------------------------")

  Model Name           | Exact Match Ratio Score
-------------------------------------------
  knn                  | {0.10144927536231885}
-------------------------------------------
  Decision Tree        | {0.2463768115942029}
-------------------------------------------
  Bagging              | {0.18840579710144928}
-------------------------------------------
  Random Forest        | {0.2028985507246377}
-------------------------------------------
  Boosting             | {0.2028985507246377}
-------------------------------------------
  Multinomial NB       | {0.014492753623188406}
-------------------------------------------
  SVC Sq. Hinge Loss   | {0.057971014492753624}
-------------------------------------------
  Power Set SVC        | {0.34782608695652173}
-------------------------------------------


In [782]:
y_train = df_train.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)
y_val = df_val.drop(['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_roles', 'entity_paragraph'], axis=1)

In [783]:
# Convert powerSetSVCPreds to a DataFrame with the same column names as y_val
powerSetSVCPreds_df = pd.DataFrame(np.array(powerSetSVCPreds.todense()), columns=y_val.columns)

In [784]:
powerSetSVCPreds_df.head()

,Bigot,Conspirator,Corrupt,Deceiver,Exploited,Foreign Adversary,Forgotten,Guardian,Incompetent,Instigator,Martyr,Peacemaker,Rebel,Saboteur,Scapegoat,Spy,Terrorist,Traitor,Tyrant,Underdog,Victim,Virtuous
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [785]:
powerSetSVCPreds_test = powerSetSVC.predict(X_test)

In [786]:
# Convert powerSetSVCPreds to a DataFrame with the same column names as y_val
powerSetSVCPreds_df_test = pd.DataFrame(np.array(powerSetSVCPreds_test.todense()), columns=y_val.columns)

In [787]:
df_test.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...


In [788]:
# Function to convert each row into a comma-separated string of column names where the value is 1
def get_column_names_with_ones(row):
    return ','.join([col for col in row.index if row[col] == 1])

# Apply the function to each row of the DataFrame
powerSetSVCPreds_df_test_single_column = powerSetSVCPreds_df_test.apply(get_column_names_with_ones, axis=1)

# Convert the result to a DataFrame with a single column
powerSetSVCPreds_df_test_single_column = pd.DataFrame(powerSetSVCPreds_df_test_single_column, columns=['Predictions'])

In [789]:
powerSetSVCPreds_df_test_single_column.head()

,Predictions
0,Instigator
1,Corrupt
2,Rebel
3,Tyrant
4,Tyrant


In [790]:
# Rename the Predictions column to the desired name, e.g., 'Labels'
powerSetSVCPreds_df_test_single_column.rename(columns={'Predictions': 'fine_grained_roles'}, inplace=True)

# Print the result to verify
powerSetSVCPreds_df_test_single_column.head()

,fine_grained_roles
0,Instigator
1,Corrupt
2,Rebel
3,Tyrant
4,Tyrant


In [791]:
# Define the role lists
PROTAGONISTS = ['Guardian', 'Martyr', 'Peacemaker', 'Rebel', 'Underdog', 'Virtuous']
ANTAGONISTS = ['Instigator', 'Conspirator', 'Tyrant', 'Foreign Adversary', 'Traitor', 'Spy', 'Saboteur', 'Corrupt', 'Incompetent', 'Terrorist', 'Deceiver', 'Bigot']
INNOCENTS = ['Forgotten', 'Exploited', 'Victim', 'Scapegoat']

# Initialize an empty list to store indices of ambiguous rows
ambiguous = []

# Function to decide the main role based on the labels
def decide_main_role(labels):
    # Split the comma-separated labels
    roles = labels.split(',')

    # Create counters for each category
    protagonist_count = sum(1 for role in roles if role in PROTAGONISTS)
    antagonist_count = sum(1 for role in roles if role in ANTAGONISTS)
    innocent_count = sum(1 for role in roles if role in INNOCENTS)

    # If more than one category is represented, check for ambiguity
    if (protagonist_count > 0 and antagonist_count > 0) or \
       (protagonist_count > 0 and innocent_count > 0) or \
       (antagonist_count > 0 and innocent_count > 0):
        ambiguous.append(labels)  # Add to ambiguous list
        return None  # Set main_role as None for ambiguous cases
    
    # Majority voting - select the category with the highest count
    if protagonist_count > max(antagonist_count, innocent_count):
        return 'Protagonist'
    elif antagonist_count > max(protagonist_count, innocent_count):
        return 'Antagonist'
    elif innocent_count > max(protagonist_count, antagonist_count):
        return 'Innocent'
    else:
        # Handle case where there's no majority (all are 0)
        return None

# Apply the function to create the 'main_role' column
powerSetSVCPreds_df_test_single_column['main_role'] = powerSetSVCPreds_df_test_single_column['fine_grained_roles'].apply(decide_main_role)

# Print the list of ambiguous rows (optional)
print("Ambiguous rows:", ambiguous)

powerSetSVCPreds_df_test_single_column.head()

Ambiguous rows: []


,fine_grained_roles,main_role
0,Instigator,Antagonist
1,Corrupt,Antagonist
2,Rebel,Protagonist
3,Tyrant,Antagonist
4,Tyrant,Antagonist


In [792]:
# Concatenate df_test and powerSetSVCPreds_df_test along the columns (axis=1)
combined_df = pd.concat([df_test, powerSetSVCPreds_df_test_single_column], axis=1)

# Print the result to verify
combined_df.head()

,article_id,entity_mention,start_offset,end_offset,entity_paragraph,fine_grained_roles,main_role
0,EN_UA_DEV_100012.txt,Washington,1441,1450,washington cannot afford allow russia achieve ...,Instigator,Antagonist
1,EN_UA_DEV_100012.txt,Americans,2397,2405,fate colonies always americans scoop natural r...,Corrupt,Antagonist
2,EN_CC_200053.txt,Rishi Sunak,85,95,uk prime minister rishi sunak wednesday said a...,Rebel,Protagonist
3,EN_CC_200040.txt,Extinction Rebellion,189,208,extinction rebellion activists everyone loves ...,Tyrant,Antagonist
4,EN_CC_200070.txt,Eric Garcetti,89,101,los angeles mayor eric garcetti joined global ...,Tyrant,Antagonist


In [608]:
df_dev = combined_df[['article_id', 'entity_mention', 'start_offset', 'end_offset', 
                      'main_role', 'fine_grained_roles']]

In [688]:
df_dev.head()

,article_id,entity_mention,start_offset,end_offset,main_role,fine_grained_roles
0,EN_UA_DEV_100012.txt,Washington,1441,1450,Antagonist,Instigator
1,EN_UA_DEV_100012.txt,Americans,2397,2405,Antagonist,Corrupt
2,EN_CC_200053.txt,Rishi Sunak,85,95,Protagonist,Rebel
3,EN_CC_200040.txt,Extinction Rebellion,189,208,Antagonist,Tyrant
4,EN_CC_200070.txt,Eric Garcetti,89,101,Antagonist,Tyrant


In [610]:
# Open the file in write mode
with open('semeval25_subtask_1_eng_sarfraz_24010739_iter_2.txt', 'w', encoding='utf-8') as f:
    # Write the header
    header = df_dev.columns.tolist()
    f.write('\t'.join(header) + '\n')
    
    # Iterate through each row and write the data
    for _, row in df_dev.iterrows():
        # Replace commas in fine_grained_roles with tabs
        fine_grained_roles = row['fine_grained_roles'].replace(',', '\t')
        
        # Create a list of row values
        row_values = [str(row[col]) if col != 'fine_grained_roles' else fine_grained_roles for col in df_dev.columns]
        
        # Write the row to the file, separating values with tabs
        f.write('\t'.join(row_values) + '\n')

# Print to verify
print("DataFrame saved successfully as a tab-separated file.")

DataFrame saved successfully as a tab-separated file.
